In [88]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("test") \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.memory", "4g") \
        .getOrCreate()

In [89]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_green.registerTempTable('green')

/home/andrewgallagher/Documents/Repositories/Data-Engineering-Zoomcamp/week5/venv/lib64/python3.10/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [90]:
df_green_revenue = spark.sql("""
SELECT 
    -- Revenue grouping 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS time, 
    
    SUM(total_amount) AS amount,
    COUNT(*) AS number_records
FROM
    green
WHERE lpep_pickup_datetime >= '2020-01-01 00:00:00' 
GROUP BY
    1, 2
ORDER BY 
    1, 2;
""")

In [91]:
df_green_revenue.show()

+----+-------------------+------+--------------+
|zone|               time|amount|number_records|
+----+-------------------+------+--------------+
|   1|2020-01-01 03:00:00| 155.3|             1|
|   1|2020-01-03 05:00:00|117.39|             1|
|   1|2020-01-26 10:00:00| 85.56|             1|
|   1|2020-01-29 18:00:00|108.36|             1|
|   1|2020-02-01 06:00:00|115.56|             1|
|   1|2020-02-06 07:00:00|  98.3|             1|
|   1|2020-02-16 07:00:00|  95.3|             1|
|   1|2020-02-17 13:00:00|  82.3|             1|
|   1|2020-02-26 17:00:00| 49.38|             1|
|   1|2020-03-02 15:00:00|  88.3|             1|
|   1|2020-03-11 12:00:00|103.56|             1|
|   1|2020-03-12 11:00:00|  61.1|             1|
|   1|2020-03-14 11:00:00|  16.0|             1|
|   1|2020-03-21 17:00:00| 100.3|             1|
|   1|2020-04-10 13:00:00| 150.3|             1|
|   1|2020-04-23 08:00:00| 37.85|             1|
|   1|2020-07-12 07:00:00| 79.05|             1|
|   1|2020-07-15 06:

In [92]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode="overwrite")

In [93]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [94]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Revenue grouping 
    PULocationID AS zone,
    date_trunc('hour', tpep_pickup_datetime) AS time, 
    
    SUM(total_amount) AS amount,
    COUNT(*) AS number_records
FROM
    yellow
WHERE tpep_pickup_datetime >= '2020-01-01 00:00:00' 
GROUP BY
    1, 2
ORDER BY 
    1, 2;
""")

In [95]:
df_green_revenue.show()

+----+-------------------+------+--------------+
|zone|               time|amount|number_records|
+----+-------------------+------+--------------+
|   1|2020-01-01 03:00:00| 155.3|             1|
|   1|2020-01-03 05:00:00|117.39|             1|
|   1|2020-01-26 10:00:00| 85.56|             1|
|   1|2020-01-29 18:00:00|108.36|             1|
|   1|2020-02-01 06:00:00|115.56|             1|
|   1|2020-02-06 07:00:00|  98.3|             1|
|   1|2020-02-16 07:00:00|  95.3|             1|
|   1|2020-02-17 13:00:00|  82.3|             1|
|   1|2020-02-26 17:00:00| 49.38|             1|
|   1|2020-03-02 15:00:00|  88.3|             1|
|   1|2020-03-11 12:00:00|103.56|             1|
|   1|2020-03-12 11:00:00|  61.1|             1|
|   1|2020-03-14 11:00:00|  16.0|             1|
|   1|2020-03-21 17:00:00| 100.3|             1|
|   1|2020-04-10 13:00:00| 150.3|             1|
|   1|2020-04-23 08:00:00| 37.85|             1|
|   1|2020-07-12 07:00:00| 79.05|             1|
|   1|2020-07-15 06:

In [96]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode="overwrite")

In [97]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'amount_green') \
    .withColumnRenamed('number_records', 'number_records_green')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'amount_yellow') \
    .withColumnRenamed('number_records', 'number_records_yellow')

In [98]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['time', 'zone'], how='outer')
df_join.show()

+-------------------+----+------------+--------------------+------------------+---------------------+
|               time|zone|amount_green|number_records_green|     amount_yellow|number_records_yellow|
+-------------------+----+------------+--------------------+------------------+---------------------+
|2020-01-01 06:00:00|   1|        null|                null|445.28000000000003|                    4|
|2020-01-01 07:00:00|   1|        null|                null|              0.31|                    1|
|2020-01-01 13:00:00|   1|        null|                null|            213.36|                    2|
|2020-01-03 06:00:00|   1|        null|                null|            174.36|                    1|
|2020-01-04 15:00:00|   1|        null|                null|            460.06|                    6|
|2020-01-05 08:00:00|   1|        null|                null|            221.76|                    2|
|2020-01-05 18:00:00|   1|        null|                null|            302.56|   

In [99]:
df_join \
    .write \
    .parquet('data/report/revenue/total', mode='overwrite')

In [100]:
df_join = spark.read.parquet('data/report/revenue/total')

In [102]:
df_zones = spark \
    .read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv('data/raw/zones/')

In [103]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [104]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [111]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID, how='inner')
df_result = df_result.drop("LocationID", "zone")
df_result.show()

+-------------------+------------+--------------------+------------------+---------------------+-------+------------+
|               time|amount_green|number_records_green|     amount_yellow|number_records_yellow|Borough|service_zone|
+-------------------+------------+--------------------+------------------+---------------------+-------+------------+
|2020-01-01 20:00:00|        null|                null|             96.96|                    3|    EWR|         EWR|
|2020-01-02 01:00:00|        null|                null|             84.36|                    1|    EWR|         EWR|
|2020-01-02 21:00:00|        null|                null|             100.3|                    1|    EWR|         EWR|
|2020-01-07 02:00:00|        null|                null|              46.8|                    1|    EWR|         EWR|
|2020-01-13 22:00:00|        null|                null|            125.35|                    1|    EWR|         EWR|
|2020-01-14 06:00:00|        null|                null|2

In [113]:
df_result \
    .write \
    .parquet('data/tmp', mode='overwrite')